In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import re
import pandas as pd
import string
import spacy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#!pip install wordcloud
from wordcloud import WordCloud

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# sStart here

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_historic_features/3/historic_retweeter_features_msc_FINAL_NEW_withpoliteness_subjectivity_v3.csv')
df

,Unnamed: 0,Unnamed: 0.1,id,Unnamed: 0.1.1,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,0,0,1072373029517316096,0,If I was a Capitalist country I would be decon...,If I was a Capitalist country I would be decon...,if i was a capitalist country i would be decon...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.780321,0.330087
1,1,1,1072373029517316096,1,@AFP A comedian says some words. https://t.co/...,A comedian says some words,a comedian says some words,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.233776,0.000000
2,2,2,1072373029517316096,2,RT @IanByrneMP: https://t.co/8uVYiFqQa5,NaN,NaN,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.437029,0.000000
3,3,3,1072373029517316096,3,@KevinPascoe @DailyMirror No surprise there Ke...,No surprise there Kev Milica Cosic joins the M...,no surprise there kev milica cosic joins the m...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.248371,0.000000
4,4,4,1072373029517316096,4,RT @KevinPascoe: @SundaySocialist @DailyMirror...,I despair A paper that once employed John Pilg...,i despair a paper that once employed john pilg...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.106440,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121121,121121,121121,1440330790101929998,121121,@mrjamesob @PieroPasset ..oh I don’t know.Some...,oh I don t know Some gregariously stupid or ev...,oh i don t know some gregariously stupid or ev...,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.038969,0.833333
121122,121122,121122,1440330790101929998,121122,@TobiFrenzen @ChillaxBcn They will arrange ano...,They will arrange another safe seat for him We...,they will arrange another safe seat for him we...,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.530612,0.500000
121123,121123,121123,1440330790101929998,121123,@CanalRiverTrust ..looks like a wee boatie on ...,looks like a wee boatie on a canal,looks like a wee boatie on a canal,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.941455,0.000000
121124,121124,121124,1440330790101929998,121124,@1goodtern @E_catservant ..and why are you at ...,catservant and why are you at all surprised,catservant and why are you at all surprised,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.055802,0.900000


In [ ]:
# other df for retweeters


df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_retweeter_features_msc_FINAL_NEW_withpoliteness_subjectivity.csv')
df2.id = df2.id.map(int)
df2

,Unnamed: 0,Unnamed: 0.1,id,Unnamed: 0.1.1,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,0,0,48959540,0,RT @T411i_: Rishi Sunak has just called all th...,Rishi Sunak has just called all the hard worki...,rishi sunak has just called all the hard worki...,0.09506,0.79719,0.07776,...,0.007694,0.489637,0.049660,0.005699,0.375193,0.019096,negative,12.0,0.028079,0.692460
1,1,1,48959540,1,RT @Heccles94: Solidarity with all striking te...,Solidarity with all striking teachers today Te...,solidarity with all striking teachers today te...,0.09506,0.79719,0.07776,...,0.007694,0.489637,0.049660,0.005699,0.375193,0.019096,negative,12.0,0.219121,1.000000
2,2,2,48959540,2,RT @rmpnco: Odd. 2 years ago I was an NHS hero...,Odd 2 years ago I was an NHS hero now those of...,odd years ago i was an nhs hero now those of u...,0.09506,0.79719,0.07776,...,0.007694,0.489637,0.049660,0.005699,0.375193,0.019096,negative,12.0,0.024563,0.204167
3,3,3,48959540,3,@PublicPriestley absolutely this!! 🔼🔼,absolutely this,absolutely this,0.09506,0.79719,0.07776,...,0.007694,0.489637,0.049660,0.005699,0.375193,0.019096,negative,12.0,0.194949,0.900000
4,4,4,48959540,4,RT @NigelHeal: @PublicPriestley absolutely thi...,absolutely this,absolutely this,0.09506,0.79719,0.07776,...,0.007694,0.489637,0.049660,0.005699,0.375193,0.019096,negative,12.0,0.194949,0.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458059,458059,458059,1440330790101929984,458059,@mrjamesob @PieroPasset ..oh I don’t know.Some...,oh I don t know Some gregariously stupid or ev...,oh i don t know some gregariously stupid or ev...,0.11315,0.77326,0.11362,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.038969,0.833333
458060,458060,458060,1440330790101929984,458060,@TobiFrenzen @ChillaxBcn They will arrange ano...,They will arrange another safe seat for him We...,they will arrange another safe seat for him we...,0.11315,0.77326,0.11362,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.530612,0.500000
458061,458061,458061,1440330790101929984,458061,@CanalRiverTrust ..looks like a wee boatie on ...,looks like a wee boatie on a canal,looks like a wee boatie on a canal,0.11315,0.77326,0.11362,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.941455,0.000000
458062,458062,458062,1440330790101929984,458062,@1goodtern @E_catservant ..and why are you at ...,catservant and why are you at all surprised,catservant and why are you at all surprised,0.11315,0.77326,0.11362,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.055802,0.900000


In [ ]:
print(df.columns)
print(df2.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'Unnamed: 0.1.1', 'text',
       'grammartext', 'clean_text', 'user_neg_mean', 'user_neu_mean',
       'user_pos_mean', 'user_comp_mean', 'user_arts_&_culture_mean',
       'user_business_&_entrepreneurs_mean',
       'user_celebrity_&_pop_culture_mean', 'user_diaries_&_daily_life_mean',
       'user_family_mean', 'user_fashion_&_style_mean',
       'user_film_tv_&_video_mean', 'user_fitness_&_health_mean',
       'user_food_&_dining_mean', 'user_gaming_mean',
       'user_learning_&_educational_mean', 'user_music_mean',
       'user_news_&_social_concern_mean', 'user_other_hobbies_mean',
       'user_relationships_mean', 'user_science_&_technology_mean',
       'user_sports_mean', 'user_travel_&_adventure_mean',
       'user_youth_&_student_life_mean', 'user_hateful_mean',
       'user_targeted_mean', 'user_aggressive_mean', 'user_joy_mean',
       'user_sadness_mean', 'user_others_mean', 'user_anger_mean',
       'user_surprise_mean', 'user_

In [ ]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1', ], axis=1)
df2 = df2.drop(['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1', ], axis=1)

In [ ]:
df.columns

Index(['id', 'text', 'grammartext', 'clean_text', 'user_neg_mean',
       'user_neu_mean', 'user_pos_mean', 'user_comp_mean',
       'user_arts_&_culture_mean', 'user_business_&_entrepreneurs_mean',
       'user_celebrity_&_pop_culture_mean', 'user_diaries_&_daily_life_mean',
       'user_family_mean', 'user_fashion_&_style_mean',
       'user_film_tv_&_video_mean', 'user_fitness_&_health_mean',
       'user_food_&_dining_mean', 'user_gaming_mean',
       'user_learning_&_educational_mean', 'user_music_mean',
       'user_news_&_social_concern_mean', 'user_other_hobbies_mean',
       'user_relationships_mean', 'user_science_&_technology_mean',
       'user_sports_mean', 'user_travel_&_adventure_mean',
       'user_youth_&_student_life_mean', 'user_hateful_mean',
       'user_targeted_mean', 'user_aggressive_mean', 'user_joy_mean',
       'user_sadness_mean', 'user_others_mean', 'user_anger_mean',
       'user_surprise_mean', 'user_disgust_mean', 'user_fear_mean',
       'user_sentiment

In [ ]:
len(df2)

458064

In [ ]:
len(df)

121126

In [ ]:
import re

text = u'This is a smiley face \U0001F5FF'
print(text) # with emoji

# def deEmojify(text):
#     regrex_pattern = re.compile(pattern = "["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#                            "]+", flags = re.UNICODE)
#     return regrex_pattern.sub(r'',text)

import re
def deEmojify(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    return tweet

def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

This is a smiley face 🗿


In [ ]:
df

,id,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,user_comp_mean,user_arts_&_culture_mean,user_business_&_entrepreneurs_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,1072373029517316096,If I was a Capitalist country I would be decon...,If I was a Capitalist country I would be decon...,if i was a capitalist country i would be decon...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.780321,0.330087
1,1072373029517316096,@AFP A comedian says some words. https://t.co/...,A comedian says some words,a comedian says some words,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.233776,0.000000
2,1072373029517316096,RT @IanByrneMP: https://t.co/8uVYiFqQa5,NaN,NaN,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.437029,0.000000
3,1072373029517316096,@KevinPascoe @DailyMirror No surprise there Ke...,No surprise there Kev Milica Cosic joins the M...,no surprise there kev milica cosic joins the m...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.248371,0.000000
4,1072373029517316096,RT @KevinPascoe: @SundaySocialist @DailyMirror...,I despair A paper that once employed John Pilg...,i despair a paper that once employed john pilg...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.106440,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121121,1440330790101929998,@mrjamesob @PieroPasset ..oh I don’t know.Some...,oh I don t know Some gregariously stupid or ev...,oh i don t know some gregariously stupid or ev...,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.038969,0.833333
121122,1440330790101929998,@TobiFrenzen @ChillaxBcn They will arrange ano...,They will arrange another safe seat for him We...,they will arrange another safe seat for him we...,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.530612,0.500000
121123,1440330790101929998,@CanalRiverTrust ..looks like a wee boatie on ...,looks like a wee boatie on a canal,looks like a wee boatie on a canal,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.941455,0.000000
121124,1440330790101929998,@1goodtern @E_catservant ..and why are you at ...,catservant and why are you at all surprised,catservant and why are you at all surprised,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.055802,0.900000


In [ ]:
not_english = []
dfcopy = df.copy()

In [ ]:
len(dfcopy)

121126

In [ ]:

# print(isEnglish("RT @moreiraviw: Esse plano básico com anúncios da Netflix é tipo"))
for i in range(len(dfcopy.text.values)):

    no_emoji = ""
    
    no_emoji = deEmojify(dfcopy.text.values[i])
    # no_emoji = clean(dfcopy.text.values[i], no_emoji=True)
    
    no_emoji = re.compile('RT').sub('', no_emoji, count=1)
    no_emoji = remove_usernames_links(no_emoji)
    no_emoji = no_emoji.replace('•', '').replace('-', '')
    # print(no_emoji)
    # no_emoji = re.sub(r'http\S+', '', no_emoji)
    no_emoji = no_emoji.replace(' ⁦ ⁦ ', '').replace(" ⁦⁦ ", " ").replace("Fernández", "Fernandez")
    no_emoji = no_emoji.strip()
    no_emoji = no_emoji.replace(',','').replace("´","").replace("¿","").replace("3⃣","").replace("Pokémon","Pokemon").replace("¥","")
    no_emoji = no_emoji.replace("4⃣","").replace("5⃣","").replace("7⃣","").replace('®','').replace("⏬","").replace("₹","").replace("¹","").replace('\u2066','').replace('\u2069','')
    no_emoji = no_emoji.replace("«","").replace("»","").replace("?","").replace("↙","").replace("°","").replace("6⃣","").replace("0⃣","")
    no_emoji = no_emoji.replace("|","").replace("„","").replace("1⃣","").replace("⏲","").replace("2⃣","").replace("⁉","").replace("⌘","").replace('\n', ' ')
    no_emoji = no_emoji.replace("⏳","").replace("$","").replace("⌛","").replace(" ًً","").replace("..."," ").replace("₦", "").replace("–","").replace("—", "").replace("™","").replace("‌‌ #","")
    no_emoji = no_emoji.replace("‼","").replace("→", '').replace('…', ' ').replace("”", "'").replace("’", "'").replace("“","").replace("‘","").replace("&amp;", "").replace("£", "")
    no_emoji = no_emoji.replace("―","").replace("€", "").replace("⏰", "").replace("⏱","").replace("∼", "").replace(" ~", " ").replace("√", "").replace("ℹ", "").replace("⏫","").replace("≤","").replace("‐", " ")
    no_emoji = no_emoji.replace(" ⁦ ","").replace("   "," ").strip().replace("~   ", " ")
    update = no_emoji
    if isEnglish(update) == False:
        not_english.append(i)
        print(update)


Streaming output truncated to the last 5000 lines.
تایملاین از دست مبارزا اگر در بیاد و‌ بره سمت مسائل حاشیه ای و تفرقه ای و فقط به نفع رژیم است چون حواس ها از خیابان و #
هر دو عزیزی که از ما کشتند توسط #دوربین‌های_شهری یا مداربسته شخصی شناسایی شده بودند. این یعنی ضروری است این انقلاب فازبندی
چقدر گفته شد که تجمع مقطعی سم است و دلایلش هم ذکر شد که یک دلیل آن دادن وقت به رژیم و سرکوبگرانش بود اما اصلاح طلبان آشکا
بیت عنکبوت خط قرمز عده ای است و سعی می کنند که حواس مردم را از آنجا و سایر مکانهای مهم پرت کنند مثلا با بولد کردن جاهای ه
هشتک رو واسه کی میزنیم؟ جمهوری اسلامی،سازمان‌های حقوق بشری یا شاید کشورهای غربی؟ از کی میخوای اعدام نکنه؟ فراخوان یعنی چ
توئیتر فارسی بجای فراخوان فقط درگیر حاشیه شده  مثل همیشه توئیتری ها وقتی به خودشون میان که التهاب جامعه فروکش کرده  فر
همه چیز حاشیه است به جز فراخوان تظاهرات  ۲۸و۲۹و۳۰آذر #مهسا_امینی
چشم شیطان در یکی از اتوبان های تهران نورانی شد  این چشم های شیطان باید در همه جای ایران از بین بروند  #مهسا_امینی #اعتصاب
یکی از روش های عملیات روانی رژیم فش

In [ ]:
# if isEnglish("Budget cuts will almost certainly follow - smart analysis as always from ") == False:
#     print("not english")

In [ ]:
len(not_english) #67866 #21380

21380

In [ ]:
dfcopy.text[2917]

'Budget cuts will almost certainly follow - smart analysis as always from \u2066@adam_tooze\u2069  https://t.co/YFjQ9YTH2R'

In [ ]:
text = u'This is a smiley face \u2069'
print(text)

This is a smiley face ⁩


In [ ]:
not_english[:20]

[63,
 70,
 73,
 74,
 75,
 76,
 167,
 177,
 705,
 1177,
 1969,
 2168,
 2372,
 2400,
 2494,
 2578,
 2676,
 2917,
 2927,
 2959]

In [ ]:
for i in not_english:
    for j in dfcopy.columns[4:]:
        dfcopy.loc[i,j]=0

In [ ]:
dfcopy.to_csv('/content/drive/MyDrive/Colab Notebooks/final_year_project/run_historic_features/check_lang/historic_retweeter_features_msc_FINAL_changed_nonenglish_v3.csv')

In [ ]:
not_english_v3 = []
df2copy = df2.copy()

In [ ]:
# print(isEnglish("RT @moreiraviw: Esse plano básico com anúncios da Netflix é tipo"))
for i in range(len(dfcopy.text.values)):

    no_emoji = ""
    
    no_emoji = deEmojify(df2copy.text.values[i])
    # no_emoji = clean(df2copy.text.values[i], no_emoji=True)
    
    no_emoji = re.compile('RT').sub('', no_emoji, count=1)
    no_emoji = remove_usernames_links(no_emoji)
    no_emoji = no_emoji.replace('•', '').replace('-', '')
    # print(no_emoji)
    # no_emoji = re.sub(r'http\S+', '', no_emoji)
    no_emoji = no_emoji.replace(' ⁦ ⁦ ', '').replace(" ⁦⁦ ", " ").replace("Fernández", "Fernandez")
    no_emoji = no_emoji.strip()
    no_emoji = no_emoji.replace(',','').replace("´","").replace("¿","").replace("3⃣","").replace("Pokémon","Pokemon").replace("¥","")
    no_emoji = no_emoji.replace("4⃣","").replace("5⃣","").replace("7⃣","").replace('®','').replace("⏬","").replace("₹","").replace("¹","").replace('\u2066','').replace('\u2069','')
    no_emoji = no_emoji.replace("«","").replace("»","").replace("?","").replace("↙","").replace("°","").replace("6⃣","").replace("0⃣","")
    no_emoji = no_emoji.replace("|","").replace("„","").replace("1⃣","").replace("⏲","").replace("2⃣","").replace("⁉","").replace("⌘","").replace('\n', ' ')
    no_emoji = no_emoji.replace("⏳","").replace("$","").replace("⌛","").replace(" ًً","").replace("..."," ").replace("₦", "").replace("–","").replace("—", "").replace("™","").replace("‌‌ #","")
    no_emoji = no_emoji.replace("‼","").replace("→", '').replace('…', ' ').replace("”", "'").replace("’", "'").replace("“","").replace("‘","").replace("&amp;", "").replace("£", "")
    no_emoji = no_emoji.replace("―","").replace("€", "").replace("⏰", "").replace("⏱","").replace("∼", "").replace(" ~", " ").replace("√", "").replace("ℹ", "").replace("⏫","").replace("≤","").replace("‐", " ")
    no_emoji = no_emoji.replace(" ⁦ ","").replace("   "," ").strip().replace("~   ", " ")
    update = no_emoji
    if isEnglish(update) == False:
        not_english_v3.append(i)
        print(update)


Streaming output truncated to the last 5000 lines.
¡Arranca! Noche de con toda la emoción y el análisis de ronda divisional y campeonatos de conferencia.
Las tomas que podremos disfrutar la próxima temporada!! Desde el casco.
Estas son las mañanitas que cantaba el Rey David  Feliz cumpleaños a  33 añitos para el piloto tapatío. Qué creen que pida de deseo
Mis amigos de  Nathaniel Hackett tomará las riendas de su ofensiva. Es en nueva OC del equipo neoyorquino. En  no la armó como entrenador en jefe. Ahora a ver qué tal
Una probadita que nos dan de las primeras imágenes de . Levante la mano quien ya quiere que se estrene.
Mamba forever!! Hace tres años sufrimos la pérdida del gran
Muchos dimes y diretes pero a ustedes amigos quién les gusta para dirigir a
En blanco!! Qué tal este diseño que subió
Casa llena!! Sus pilotos y líderes. De qué tripleta esperan más
El primer día de  con
Recordando las travesuras de Manning! Cuando  la sufrió en un Pro Bowl.
. tiene una motivación mayor para t

In [ ]:
len(not_english)

21380

In [ ]:
for i in not_english_v3:
    for j in df2copy.columns[4:]:
        df2copy.loc[i,j]=0

In [ ]:
len(df2copy)

458064

In [ ]:
df2copy.to_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_retweeter_features_msc_FINAL_changed_nonenglish.csv')

In [ ]:
len(df2copy)

458064

# End

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_features_msc_FINAL_withpoliteness_subjectivity.csv')
df

,Unnamed: 0,Unnamed: 0.1,id,Unnamed: 0.1.1,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,0,0,1327956774192893952,0,RT @ShankiAustine: That Guy on the second fram...,That Guy on the second frame actually looks li...,that guy on the second frame actually looks li...,0.065283,0.824717,0.089798,...,0.027622,0.764534,0.023244,0.025647,0.073312,0.014813,neutral,3.0,0.833741,0.050000
1,1,1,1327956774192893952,1,RT @Propesaa: Cardi B's story 22hrs ago 👀 http...,Cardi B s story 22hrs ago,cardi b s story hrs ago,0.065283,0.824717,0.089798,...,0.027622,0.764534,0.023244,0.025647,0.073312,0.014813,neutral,3.0,0.429688,0.000000
2,2,2,1327956774192893952,2,RT @DailyLoud: Man shoots gun in air and almos...,Man shoots gun in air and almost makes a fatal...,man shoots gun in air and almost makes a fatal...,0.065283,0.824717,0.089798,...,0.027622,0.764534,0.023244,0.025647,0.073312,0.014813,neutral,3.0,0.049423,0.000000
3,3,3,1327956774192893952,3,RT @254evee_: Sooner or later she will realize...,Sooner or later she will realize it,sooner or later she will realize it,0.065283,0.824717,0.089798,...,0.027622,0.764534,0.023244,0.025647,0.073312,0.014813,neutral,3.0,0.366038,0.000000
4,4,4,1327956774192893952,4,RT @arthurton_zw: Me waiting for her to have t...,Me waiting for her to have the summer body she...,me waiting for her to have the summer body she...,0.065283,0.824717,0.089798,...,0.027622,0.764534,0.023244,0.025647,0.073312,0.014813,neutral,3.0,0.386282,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430433,430433,430433,105011437,430433,RT @JamieSale: It's Time for the Scientific Co...,It s Time for the Scientific Community to Admi...,it s time for the scientific community to admi...,0.086687,0.782636,0.070040,...,0.021792,0.597312,0.019842,0.020407,0.226587,0.088920,neutral,12.0,0.164623,0.900000
430434,430434,430434,105011437,430434,RT @ChuckCallesto: BREAKING REPORT: Former Twi...,BREAKING REPORT Former Twitter Execs Yoel Roth...,breaking report former twitter execs yoel roth...,0.086687,0.782636,0.070040,...,0.021792,0.597312,0.019842,0.020407,0.226587,0.088920,neutral,12.0,0.068814,0.000000
430435,430435,430435,105011437,430435,@EBaySurfer Everyone who votes should tweet you,Everyone who votes should tweet you,everyone who votes should tweet you,0.086687,0.782636,0.070040,...,0.021792,0.597312,0.019842,0.020407,0.226587,0.088920,neutral,12.0,0.648313,0.000000
430436,430436,430436,105011437,430436,RT @UntilIcanthink: Do eggs by any chance kill...,Do eggs by any chance kill the spike protein,do eggs by any chance kill the spike protein,0.086687,0.782636,0.070040,...,0.021792,0.597312,0.019842,0.020407,0.226587,0.088920,neutral,12.0,0.582906,0.000000


In [ ]:
for x in df.clean_text.values[:-1000]:
    print(x)

Streaming output truncated to the last 5000 lines.
shubman gill became only the fifth indian to score hundreds in all three formats for india shubmangill indvsnz
exclusive as india is becoming easier place to do business global stars are aligning themselves to support us minister for commerce amp industry newstrack budgetonindiatoday
ananyapanday to star in vikramadityamotwane s new film says dreams do come true
this is an outstanding budget it is disciplined growth oriented and sustainable md and ceo mahindra group newstrack budgetonindiatoday
young opener shubmangill produced an incredible display of power hitting to hit his maiden t20i hundred during the third t20i against new zealand read more here
exclusive it is not the allocation which matters but what is the actual expenditure matters in former finance minister newstrack budgetonindiatoday
india score for in overs shubman gill stays unbeaten on indvsnz live updates
we agreed to cooperate biden s personal lawyer bob bauer said i

KeyboardInterrupt: ignored

In [ ]:

len(df)

121126

In [ ]:
df

,id,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,user_comp_mean,user_arts_&_culture_mean,user_business_&_entrepreneurs_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,1072373029517316096,If I was a Capitalist country I would be decon...,If I was a Capitalist country I would be decon...,if i was a capitalist country i would be decon...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.780321,0.330087
1,1072373029517316096,@AFP A comedian says some words. https://t.co/...,A comedian says some words,a comedian says some words,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.233776,0.000000
2,1072373029517316096,RT @IanByrneMP: https://t.co/8uVYiFqQa5,NaN,NaN,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.437029,0.000000
3,1072373029517316096,@KevinPascoe @DailyMirror No surprise there Ke...,No surprise there Kev Milica Cosic joins the M...,no surprise there kev milica cosic joins the m...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.248371,0.000000
4,1072373029517316096,RT @KevinPascoe: @SundaySocialist @DailyMirror...,I despair A paper that once employed John Pilg...,i despair a paper that once employed john pilg...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.106440,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121121,1440330790101929998,@mrjamesob @PieroPasset ..oh I don’t know.Some...,oh I don t know Some gregariously stupid or ev...,oh i don t know some gregariously stupid or ev...,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.038969,0.833333
121122,1440330790101929998,@TobiFrenzen @ChillaxBcn They will arrange ano...,They will arrange another safe seat for him We...,they will arrange another safe seat for him we...,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.530612,0.500000
121123,1440330790101929998,@CanalRiverTrust ..looks like a wee boatie on ...,looks like a wee boatie on a canal,looks like a wee boatie on a canal,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.941455,0.000000
121124,1440330790101929998,@1goodtern @E_catservant ..and why are you at ...,catservant and why are you at all surprised,catservant and why are you at all surprised,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.055802,0.900000


In [ ]:
df.columns

Index(['id', 'text', 'grammartext', 'clean_text', 'user_neg_mean',
       'user_neu_mean', 'user_pos_mean', 'user_comp_mean',
       'user_arts_&_culture_mean', 'user_business_&_entrepreneurs_mean',
       'user_celebrity_&_pop_culture_mean', 'user_diaries_&_daily_life_mean',
       'user_family_mean', 'user_fashion_&_style_mean',
       'user_film_tv_&_video_mean', 'user_fitness_&_health_mean',
       'user_food_&_dining_mean', 'user_gaming_mean',
       'user_learning_&_educational_mean', 'user_music_mean',
       'user_news_&_social_concern_mean', 'user_other_hobbies_mean',
       'user_relationships_mean', 'user_science_&_technology_mean',
       'user_sports_mean', 'user_travel_&_adventure_mean',
       'user_youth_&_student_life_mean', 'user_hateful_mean',
       'user_targeted_mean', 'user_aggressive_mean', 'user_joy_mean',
       'user_sadness_mean', 'user_others_mean', 'user_anger_mean',
       'user_surprise_mean', 'user_disgust_mean', 'user_fear_mean',
       'user_sentiment

In [ ]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1', ], axis=1)

In [ ]:

df.clean_text.head()

0    if i was a capitalist country i would be decon...
1                           a comedian says some words
2                                                  NaN
3    no surprise there kev milica cosic joins the m...
4    i despair a paper that once employed john pilg...
Name: clean_text, dtype: object

In [ ]:
dfcopy = df.copy()

In [ ]:
not_english = []
not_english_id = []

In [ ]:
for i in range(len(dfcopy.clean_text.values)):
    # x = re.sub(r'http\S+', '', x)
    # x =  re.sub('@[^\s]+','',x)
    # print(x)
    # print(x)
    try: 
        detect(str(dfcopy.clean_text.values[i]))
        if detect(str(dfcopy.clean_text.values[i])) != 'en':
            print(dfcopy.id[i] , dfcopy.text.values[i])
            not_english.append(df.id[i])
            not_english_id.append(i)

            # dfcopy.drop(dfcopy.index[i],axis=0)

    except:
        print(dfcopy.id[i] , dfcopy.text.values[i])
        not_english.append(dfcopy.id[i])
        not_english_id.append(i)

Streaming output truncated to the last 5000 lines.
1499133763372167173 RT @katalin_pota: Good morning friends!💛 https://t.co/btztteqkQm
1499133763372167173 @yannii1973 mish  ufo
1499133763372167173 @yannii1973 two        aiwa
1499133763372167173 RT @blackdiammon: ‘W’ 👍 https://t.co/05uWsAJcP1
1499133763372167173 @JeanCoombes2 @StephenKing You mean stephen king    ok..  now i understand
1499133763372167173 RT @3sbahaj: صباح الخير والخيرات
وش يعني سيكوفايف؟
1499133763372167173 @3sbahaj wa sabah  al khair   ya salem
1499133763372167173 @3sbahaj sabah al  khair  ya  salem
1499133763372167173 @vivekwith00 Yes
1499133763372167173 @Tushar_KN Tushar  ji suprabhat
1499133763372167173 Nadhim zahawi     sacked  by  rishi  sunak  pm   uk government
1499133763372167173 @3sbahaj Ya sukran   ya  salem   inshalla  inta  bi kheir  fee   al yom
1499133763372167173 RT @3sbahaj: مسيرات مجهوله تضرب مواقع عسكريه ايرانية
قد تكون المسيرات تبع
إسرائيل 🇮🇱 تمثيلية
أوكرانيا 🇺🇦 دعم روسيا بالمسيرات 
أذربيجان 🇦🇿 ع…


In [ ]:
len(not_english )

46024

In [ ]:
len(not_english_id)

46024

In [ ]:
not_english[0]

1072373029517316096

In [ ]:
not_english_id[:10]

[2, 8, 21, 25, 60, 63, 65, 67, 99, 101]

In [ ]:
len(df)

121126

In [ ]:
len(dfcopy)

In [ ]:
len(df)

In [ ]:
dfnew = pd.DataFrame(list(zip(not_english_id, not_english)),
              columns=['indexes','id'])

In [ ]:
dfnew.to_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/test_retweeter_not_english.csv')

In [ ]:
dfnew

,index,id
0,2,1072373029517316096
1,8,1072373029517316096
2,21,1072373029517316096
3,25,1072373029517316096
4,60,1072373029517316096
...,...,...
46019,121040,1440330790101929998
46020,121052,1440330790101929998
46021,121073,1440330790101929998
46022,121074,1440330790101929998


In [ ]:
mergeddf=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_retweeter_features_msc_FINAL_NEW_withpoliteness_subjectivity_v3.csv')

In [ ]:
mergeddf

,Unnamed: 0,Unnamed: 0.1,id,Unnamed: 0.1.1,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,0,0,1072373029517316096,0,If I was a Capitalist country I would be decon...,If I was a Capitalist country I would be decon...,if i was a capitalist country i would be decon...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.780321,0.330087
1,1,1,1072373029517316096,1,@AFP A comedian says some words. https://t.co/...,A comedian says some words,a comedian says some words,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.233776,0.000000
2,2,2,1072373029517316096,2,RT @IanByrneMP: https://t.co/8uVYiFqQa5,NaN,NaN,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.437029,0.000000
3,3,3,1072373029517316096,3,@KevinPascoe @DailyMirror No surprise there Ke...,No surprise there Kev Milica Cosic joins the M...,no surprise there kev milica cosic joins the m...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.248371,0.000000
4,4,4,1072373029517316096,4,RT @KevinPascoe: @SundaySocialist @DailyMirror...,I despair A paper that once employed John Pilg...,i despair a paper that once employed john pilg...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.106440,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121121,121121,121121,1440330790101929998,121121,@mrjamesob @PieroPasset ..oh I don’t know.Some...,oh I don t know Some gregariously stupid or ev...,oh i don t know some gregariously stupid or ev...,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.038969,0.833333
121122,121122,121122,1440330790101929998,121122,@TobiFrenzen @ChillaxBcn They will arrange ano...,They will arrange another safe seat for him We...,they will arrange another safe seat for him we...,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.530612,0.500000
121123,121123,121123,1440330790101929998,121123,@CanalRiverTrust ..looks like a wee boatie on ...,looks like a wee boatie on a canal,looks like a wee boatie on a canal,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.941455,0.000000
121124,121124,121124,1440330790101929998,121124,@1goodtern @E_catservant ..and why are you at ...,catservant and why are you at all surprised,catservant and why are you at all surprised,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.055802,0.900000


In [ ]:
dfnew


,indexes,id
0,2,1072373029517316096
1,8,1072373029517316096
2,21,1072373029517316096
3,25,1072373029517316096
4,60,1072373029517316096
...,...,...
46019,121040,1440330790101929998
46020,121052,1440330790101929998
46021,121073,1440330790101929998
46022,121074,1440330790101929998


In [ ]:
lst = dfnew.indexes.values.tolist()

In [ ]:
lst[:10]

[2, 8, 21, 25, 60, 63, 65, 67, 99, 101]

In [ ]:
len(lst)

46024

In [ ]:
mergeddf = mergeddf.drop(index=lst)
# mergeddf = mergeddf
mergeddf

,Unnamed: 0,Unnamed: 0.1,id,Unnamed: 0.1.1,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,0,0,1072373029517316096,0,If I was a Capitalist country I would be decon...,If I was a Capitalist country I would be decon...,if i was a capitalist country i would be decon...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.780321,0.330087
1,1,1,1072373029517316096,1,@AFP A comedian says some words. https://t.co/...,A comedian says some words,a comedian says some words,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.233776,0.000000
3,3,3,1072373029517316096,3,@KevinPascoe @DailyMirror No surprise there Ke...,No surprise there Kev Milica Cosic joins the M...,no surprise there kev milica cosic joins the m...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.248371,0.000000
4,4,4,1072373029517316096,4,RT @KevinPascoe: @SundaySocialist @DailyMirror...,I despair A paper that once employed John Pilg...,i despair a paper that once employed john pilg...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.106440,0.000000
5,5,5,1072373029517316096,5,@BenWill1973 @Roppa77 @UKLabour How Tory infes...,How Tory infested Push Polling will trash at t...,how tory infested push polling will trash at t...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.341041,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121121,121121,121121,1440330790101929998,121121,@mrjamesob @PieroPasset ..oh I don’t know.Some...,oh I don t know Some gregariously stupid or ev...,oh i don t know some gregariously stupid or ev...,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.038969,0.833333
121122,121122,121122,1440330790101929998,121122,@TobiFrenzen @ChillaxBcn They will arrange ano...,They will arrange another safe seat for him We...,they will arrange another safe seat for him we...,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.530612,0.500000
121123,121123,121123,1440330790101929998,121123,@CanalRiverTrust ..looks like a wee boatie on ...,looks like a wee boatie on a canal,looks like a wee boatie on a canal,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.941455,0.000000
121124,121124,121124,1440330790101929998,121124,@1goodtern @E_catservant ..and why are you at ...,catservant and why are you at all surprised,catservant and why are you at all surprised,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.055802,0.900000


In [ ]:
mergeddf.reset_index(drop=True)

,Unnamed: 0,Unnamed: 0.1,id,Unnamed: 0.1.1,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,0,0,1072373029517316096,0,If I was a Capitalist country I would be decon...,If I was a Capitalist country I would be decon...,if i was a capitalist country i would be decon...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.780321,0.330087
1,1,1,1072373029517316096,1,@AFP A comedian says some words. https://t.co/...,A comedian says some words,a comedian says some words,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.233776,0.000000
2,3,3,1072373029517316096,3,@KevinPascoe @DailyMirror No surprise there Ke...,No surprise there Kev Milica Cosic joins the M...,no surprise there kev milica cosic joins the m...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.248371,0.000000
3,4,4,1072373029517316096,4,RT @KevinPascoe: @SundaySocialist @DailyMirror...,I despair A paper that once employed John Pilg...,i despair a paper that once employed john pilg...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.106440,0.000000
4,5,5,1072373029517316096,5,@BenWill1973 @Roppa77 @UKLabour How Tory infes...,How Tory infested Push Polling will trash at t...,how tory infested push polling will trash at t...,0.107141,0.751606,0.090707,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.341041,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75097,121121,121121,1440330790101929998,121121,@mrjamesob @PieroPasset ..oh I don’t know.Some...,oh I don t know Some gregariously stupid or ev...,oh i don t know some gregariously stupid or ev...,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.038969,0.833333
75098,121122,121122,1440330790101929998,121122,@TobiFrenzen @ChillaxBcn They will arrange ano...,They will arrange another safe seat for him We...,they will arrange another safe seat for him we...,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.530612,0.500000
75099,121123,121123,1440330790101929998,121123,@CanalRiverTrust ..looks like a wee boatie on ...,looks like a wee boatie on a canal,looks like a wee boatie on a canal,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.941455,0.000000
75100,121124,121124,1440330790101929998,121124,@1goodtern @E_catservant ..and why are you at ...,catservant and why are you at all surprised,catservant and why are you at all surprised,0.113150,0.773260,0.113620,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.055802,0.900000


In [ ]:
mergeddf.to_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_retweeter_features_msc_FINAL_withpoliteness_subjectivity_english.csv')

In [ ]:
not_english = []

In [ ]:
pip install TextBlob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df.columns


Index(['id', 'Unnamed: 0', 'text', 'grammartext', 'clean_text',
       'user_neg_mean', 'user_neu_mean', 'user_pos_mean', 'user_comp_mean',
       'user_arts_&_culture_mean', 'user_business_&_entrepreneurs_mean',
       'user_celebrity_&_pop_culture_mean', 'user_diaries_&_daily_life_mean',
       'user_family_mean', 'user_fashion_&_style_mean',
       'user_film_tv_&_video_mean', 'user_fitness_&_health_mean',
       'user_food_&_dining_mean', 'user_gaming_mean',
       'user_learning_&_educational_mean', 'user_music_mean',
       'user_news_&_social_concern_mean', 'user_other_hobbies_mean',
       'user_relationships_mean', 'user_science_&_technology_mean',
       'user_sports_mean', 'user_travel_&_adventure_mean',
       'user_youth_&_student_life_mean', 'user_hateful_mean',
       'user_targeted_mean', 'user_aggressive_mean', 'user_joy_mean',
       'user_sadness_mean', 'user_others_mean', 'user_anger_mean',
       'user_surprise_mean', 'user_disgust_mean', 'user_fear_mean',
       '

In [ ]:
len(df.clean_text)

640521

In [ ]:
# df = df.drop(['Unnamed: 0', 'Unnamed: 0.1' , 'Unnamed: 0.1.1', 'level_0', 'index'], axis=1)
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
import re

not_english = []
for i in range(len(df.clean_text.values)):
    # x = re.sub(r'http\S+', '', x)
    # x =  re.sub('@[^\s]+','',x)
    # print(x)
    # print(x)
    try:
        test =  detect(str(df.clean_text.values[i]))
        if test != 'en':
            not_english.append(df.id)
            df.drop(i,axis=0)
            # df = df.reset_index()
    except:
        pass

# import re
# from textblob import TextBlob
# for x in df.text.values:
#     # print(x)
#     x = re.sub(r'http\S+', '', x)
#     b = TextBlob(x)
#     print(b)
#     print(b.detect_language())
#     if  b.detect_language() !='en':
#         not_english.append(df.id)


In [ ]:
# df2 = pd.DataFrame(not_english)
# df.to_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_tweets_features_msc_notenglish.csv')
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_tweets_features_msc_NEW_ENGLISH.csv')

In [ ]:
not_english

[0         1.072601e+18
 1         1.072601e+18
 2         1.072601e+18
 3         1.072601e+18
 4         1.072601e+18
               ...     
 640516    1.207502e+18
 640517    1.207502e+18
 640518    1.207502e+18
 640519    1.207502e+18
 640520    1.207502e+18
 Name: id, Length: 640521, dtype: float64, 0         1.072601e+18
 1         1.072601e+18
 2         1.072601e+18
 3         1.072601e+18
 4         1.072601e+18
               ...     
 640516    1.207502e+18
 640517    1.207502e+18
 640518    1.207502e+18
 640519    1.207502e+18
 640520    1.207502e+18
 Name: id, Length: 640521, dtype: float64, 0         1.072601e+18
 1         1.072601e+18
 2         1.072601e+18
 3         1.072601e+18
 4         1.072601e+18
               ...     
 640516    1.207502e+18
 640517    1.207502e+18
 640518    1.207502e+18
 640519    1.207502e+18
 640520    1.207502e+18
 Name: id, Length: 640521, dtype: float64, 0         1.072601e+18
 1         1.072601e+18
 2         1.072601e+18
 3        

In [ ]:
not_english.unique()

AttributeError: ignored

In [ ]:
import csv
with open('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/not_english_NEW', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    for val in not_english:
        write.writerow([val])

In [ ]:
df3=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_retweeter_features_msc_FINAL_NEW_withpoliteness_subjectivity_v3.csv')

In [ ]:
df3.columns


Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'Unnamed: 0.1.1', 'text',
       'grammartext', 'clean_text', 'user_neg_mean', 'user_neu_mean',
       'user_pos_mean', 'user_comp_mean', 'user_arts_&_culture_mean',
       'user_business_&_entrepreneurs_mean',
       'user_celebrity_&_pop_culture_mean', 'user_diaries_&_daily_life_mean',
       'user_family_mean', 'user_fashion_&_style_mean',
       'user_film_tv_&_video_mean', 'user_fitness_&_health_mean',
       'user_food_&_dining_mean', 'user_gaming_mean',
       'user_learning_&_educational_mean', 'user_music_mean',
       'user_news_&_social_concern_mean', 'user_other_hobbies_mean',
       'user_relationships_mean', 'user_science_&_technology_mean',
       'user_sports_mean', 'user_travel_&_adventure_mean',
       'user_youth_&_student_life_mean', 'user_hateful_mean',
       'user_targeted_mean', 'user_aggressive_mean', 'user_joy_mean',
       'user_sadness_mean', 'user_others_mean', 'user_anger_mean',
       'user_surprise_mean', 'user_

In [ ]:
df3 = df3.drop(['Unnamed: 0', 'Unnamed: 0.1' , 'Unnamed: 0.1.1'], axis=1)

In [ ]:
df3.head()

,id,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,user_comp_mean,user_arts_&_culture_mean,user_business_&_entrepreneurs_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,1072373029517316096,If I was a Capitalist country I would be decon...,If I was a Capitalist country I would be decon...,if i was a capitalist country i would be decon...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.00949,0.432934,0.035853,0.005974,0.47273,0.014999,negative,12.0,0.780321,0.330087
1,1072373029517316096,@AFP A comedian says some words. https://t.co/...,A comedian says some words,a comedian says some words,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.00949,0.432934,0.035853,0.005974,0.47273,0.014999,negative,12.0,0.233776,0.000000
2,1072373029517316096,RT @IanByrneMP: https://t.co/8uVYiFqQa5,NaN,NaN,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.00949,0.432934,0.035853,0.005974,0.47273,0.014999,negative,12.0,0.437029,0.000000
3,1072373029517316096,@KevinPascoe @DailyMirror No surprise there Ke...,No surprise there Kev Milica Cosic joins the M...,no surprise there kev milica cosic joins the m...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.00949,0.432934,0.035853,0.005974,0.47273,0.014999,negative,12.0,0.248371,0.000000
4,1072373029517316096,RT @KevinPascoe: @SundaySocialist @DailyMirror...,I despair A paper that once employed John Pilg...,i despair a paper that once employed john pilg...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.00949,0.432934,0.035853,0.005974,0.47273,0.014999,negative,12.0,0.106440,0.000000


In [ ]:
not_english2 = []

In [ ]:
df3_copy = df3.copy()

In [ ]:
df3_copy

,id,text,grammartext,clean_text,user_neg_mean,user_neu_mean,user_pos_mean,user_comp_mean,user_arts_&_culture_mean,user_business_&_entrepreneurs_mean,...,user_sadness_mean,user_others_mean,user_anger_mean,user_surprise_mean,user_disgust_mean,user_fear_mean,user_sentiment_mode,user_topic_mode,politeness,tweet_subjectivity
0,1072373029517316096,If I was a Capitalist country I would be decon...,If I was a Capitalist country I would be decon...,if i was a capitalist country i would be decon...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.780321,0.330087
1,1072373029517316096,@AFP A comedian says some words. https://t.co/...,A comedian says some words,a comedian says some words,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.233776,0.000000
2,1072373029517316096,RT @IanByrneMP: https://t.co/8uVYiFqQa5,NaN,NaN,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.437029,0.000000
3,1072373029517316096,@KevinPascoe @DailyMirror No surprise there Ke...,No surprise there Kev Milica Cosic joins the M...,no surprise there kev milica cosic joins the m...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.248371,0.000000
4,1072373029517316096,RT @KevinPascoe: @SundaySocialist @DailyMirror...,I despair A paper that once employed John Pilg...,i despair a paper that once employed john pilg...,0.107141,0.751606,0.090707,-0.030808,0.018257,0.084425,...,0.009490,0.432934,0.035853,0.005974,0.472730,0.014999,negative,12.0,0.106440,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121121,1440330790101929998,@mrjamesob @PieroPasset ..oh I don’t know.Some...,oh I don t know Some gregariously stupid or ev...,oh i don t know some gregariously stupid or ev...,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.038969,0.833333
121122,1440330790101929998,@TobiFrenzen @ChillaxBcn They will arrange ano...,They will arrange another safe seat for him We...,they will arrange another safe seat for him we...,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.530612,0.500000
121123,1440330790101929998,@CanalRiverTrust ..looks like a wee boatie on ...,looks like a wee boatie on a canal,looks like a wee boatie on a canal,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.941455,0.000000
121124,1440330790101929998,@1goodtern @E_catservant ..and why are you at ...,catservant and why are you at all surprised,catservant and why are you at all surprised,0.113150,0.773260,0.113620,-0.012402,0.032640,0.034412,...,0.015912,0.633325,0.020088,0.021851,0.264173,0.011632,negative,12.0,0.055802,0.900000


In [ ]:
for i in range(len(df3_copy.clean_text.values)):
    # x = re.sub(r'http\S+', '', x)
    # x =  re.sub('@[^\s]+','',x)
    # print(x)
    # print(x)
    if detect(str(df3_copy.clean_text.values[i])) != 'en':
        # not_english.append(df.id)
        df3_copy.drop(i,axis=0)


In [ ]:
len(df3_copy)

121126

In [ ]:
len(df3)

121126

In [ ]:
# df4 = pd.DataFrame(not_english2)
df3.to_csv('/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/raw_data/historic tweets/historic_retweeter_tweets_features_msc_notenglish.csv')